In [1]:
#import cell
import QuantLib as ql
from IPython.core.display import display, Markdown
from scipy import optimize
from collateralAgreement import CollateralAgreement
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
from marketdata.fx_spot import FxSpot
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection, TradeDirection
asdf =1

## AddOn for interest rate derivatives

#### Step 1 - calculation of effective notional $D_{jk}^{IR}$

\begin{align*}
D_{jk}^{IR} &= \sum_{i\in\left\{Ccy_j, MB_k\right\}}{\delta_i*d_i^{IR}*MF_i}
\end{align*}

Here, the notation $i\in\left\{Ccy_j, MB_k\right\}$ refers to trades whose underlying is the interest rate of a common currency $j$ and which mature in a common maturity bucket $k$

We can test our implementation of the effective notional against a small exemplary portfolio in Annex 4a Example 1 of the SA_CCR paper. It consists of the following trades:

| Trade # | Nature             | Residual Maturity | Base Ccy | Notional (tsd) | Pay Leg              | Receive Leg          | Market value (tsd)        |
|---------|--------------------|-------------------|---------------|----------------------|----------------------|--------------------------|--------------------------|
| 1       | Interest rate swap | 10 years          | USD           | 10000                | Fixed                | Floating                 | 30                       |
| 2       | Interest rate swap | 4 years           | USD           | 10000                | Floating             | Fixed                    | -20                      |
| 3       | European Swaption  | 1 into 10 years   | EUR           | 5000                 | Floating             | Fixed                    | 50                       |

To set up this exemplary portfolio we need to find fixed rates for the swaps and underlying swaps to match the desired market values.

In [2]:
def find_1(fixed_rate):
    target_value = 30000
    swap = IRS(notional = 10000000,
               timeToSwapStart=ql.Period(2, ql.Days),
               timeToSwapEnd=ql.Period(10, ql.Years),
               swapDirection=SwapDirection.PAYER,
               index = InterestRateIndex.USDLIBOR3M,
               fixed_rate=fixed_rate[0]
               )
    return abs(swap.get_price()-target_value)

def find_2(fixed_rate):
    target_value = -20000
    swap = IRS(notional = 10000000,
               timeToSwapStart=ql.Period(2, ql.Days),
               timeToSwapEnd=ql.Period(4, ql.Years),
               swapDirection=SwapDirection.RECEIVER,
               index = InterestRateIndex.USDLIBOR3M,
               fixed_rate=fixed_rate[0]
               )
    return abs(swap.get_price()-target_value)

def find_3(fixed_rate):
    target_value = 50000
    notional = 5000000*FxSpot.USDEUR.value
    swap = IRS(notional = notional,
               timeToSwapStart=ql.Period(1, ql.Years),
               timeToSwapEnd=ql.Period(10, ql.Years),
               swapDirection=SwapDirection.RECEIVER,
               index = InterestRateIndex.EURIBOR6M,
               fixed_rate=fixed_rate[0]
               )
    swaption = Swaption(underlyingSwap=swap,
                        optionMaturity=ql.Period(1, ql.Years),
                        tradeDirection=TradeDirection.LONG)
    price = swaption.get_price()*FxSpot.EURUSD.value
    return abs(price-target_value)

def find_4(fixed_rate):
    target_value = -0.27
    notional = 5000000*FxSpot.USDEUR.value
    swap = IRS(notional = notional,
               timeToSwapStart=ql.Period(1, ql.Years),
               timeToSwapEnd=ql.Period(10, ql.Years),
               swapDirection=SwapDirection.RECEIVER,
               index = InterestRateIndex.EURIBOR6M,
               fixed_rate=fixed_rate[0]
               )
    swaption = Swaption(underlyingSwap=swap,
                        optionMaturity=ql.Period(1, ql.Years),
                        tradeDirection=TradeDirection.LONG)
    delta = SA_CCR.calculate_sa_ccr_delta(swaption)
    return abs(delta-target_value)

result = optimize.minimize(find_1, x0=0.01, tol=0.0000000001)
fixed_rate_trade_1 = result.x[0]
result = optimize.minimize(find_2, x0=0.01, tol=0.0000000001)
fixed_rate_trade_2 = result.x[0]
result = optimize.minimize(find_3, x0=0.01, constraints={'type':'ineq','fun': lambda x: x[0]+0.02}, tol=0.0000000001)
fixed_rate_trade_3 = result.x[0]
result = optimize.minimize(find_4, x0=0.01, constraints={'type':'ineq','fun': lambda x: x[0]}, tol=0.0000000001)
fixed_rate_trade_4 = result.x[0]

Through optimization and using the market data of the 10th of May 2019 the fixed rates to match the market values in Example 1 were identified.

In [3]:
trade_1 = IRS(notional = 10000000,
             timeToSwapStart=ql.Period(2, ql.Days),
             timeToSwapEnd=ql.Period(10, ql.Years),
             swapDirection=SwapDirection.PAYER,
             index = InterestRateIndex.USDLIBOR3M,
             fixed_rate=fixed_rate_trade_1
             )

trade_2 = IRS(notional = 10000000,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(4, ql.Years),
              swapDirection=SwapDirection.RECEIVER,
              index = InterestRateIndex.USDLIBOR3M,
              fixed_rate=fixed_rate_trade_2
              )

notional = 5000000*FxSpot.USDEUR.value
ul_swap = IRS(notional = notional,
              timeToSwapStart=ql.Period(1, ql.Years),
              timeToSwapEnd=ql.Period(10, ql.Years),
              swapDirection=SwapDirection.RECEIVER,
              index = InterestRateIndex.EURIBOR6M,
              fixed_rate=fixed_rate_trade_3
              )

trade_3 = Swaption(underlyingSwap=ul_swap,
                   optionMaturity=ql.Period(1, ql.Years),
                   tradeDirection=TradeDirection.LONG)

test_portfolio = [trade_1, trade_2, trade_3]

In [4]:
display(Markdown('For trade 1 the matching fixed rate is %.4f%%, for trade 1 it is %.4f%% and for the underlying swap of trade 3 it is %.4f%%' % ((fixed_rate_trade_1*100),(fixed_rate_trade_2*100),(fixed_rate_trade_3*100))))

For trade 1 the matching fixed rate is 2.3754%, for trade 1 it is 2.2108% and for the underlying swap of trade 3 it is 0.1610%

In [5]:
ca=CollateralAgreement()

print(SA_CCR.calculate_sa_ccr_delta(trade_1))
print(SA_CCR.trade_level_adjusted_notional(trade_1))
print(SA_CCR.calculate_sa_ccr_delta(trade_2))
print(SA_CCR.trade_level_adjusted_notional(trade_2))
print(SA_CCR.calculate_sa_ccr_delta(trade_3))
print(SA_CCR.trade_level_adjusted_notional(trade_3))

SA_CCR.interest_rate_addOn(test_portfolio,ca)



1
78638320.21725275
-1
36198301.54418307
-0.00343382383244269
31712286.360503413


296732.72980308026

In [7]:
export("SA_CCR_ird_addon.ipynb")